In [1]:
import os
import torch
import torchvision as tv
import transformer_flow 
import utils
import pathlib

notebook_output_path = 'runs/notebook'

In [2]:
# Specify the following parameters to match the model config
dataset = 'afhq'
num_classes = {'imagenet': 1000, 'imagenet64': 0, 'afhq': 3}[dataset]
img_size = 256
channel_size = 3

batch_size = 16
patch_size = 8
channels = 768
blocks = 8
layers_per_block = 8
noise_std = 0.07

device = 'cuda'

model_name = f'{patch_size}_{channels}_{blocks}_{layers_per_block}_{noise_std:.2f}'
ckpt_file = os.path.join(notebook_output_path, f'{dataset}_model_{model_name}.pth')

sample_dir = os.path.join(notebook_output_path, f'{dataset}_samples_{model_name}')
os.makedirs(sample_dir, exist_ok=True)

model = transformer_flow.Model(in_channels=channel_size, img_size=img_size, patch_size=patch_size, 
              channels=channels, num_blocks=blocks, layers_per_block=layers_per_block,
             num_classes=num_classes).to(device)
model.load_state_dict(torch.load('/home/groups/btrippe/henry/cgm/tarflow/models/afhq_model_8_768_8_8_0.07.pth'))
print('checkpoint loaded!')

/tmp/ipykernel_28746/2185015166.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home/groups/btrippe/henry/cgm/tarflow/models/afhq_mod

checkpoint loaded!


In [4]:
fixed_noise = torch.randn(batch_size, (img_size // patch_size)**2, channel_size * patch_size ** 2, device=device)
if num_classes:
    fixed_y = torch.randint(num_classes, (batch_size,), device=device)
else:
    fixed_y = None
fixed_y = 2 * torch.ones_like(fixed_y, device=device)

In [6]:
# now let's generate samples
guided_samples = {}
guidance = 1.
attn_temp = 1.
with torch.no_grad():
    with torch.autocast(device_type=device, dtype=torch.bfloat16):
        samples = model.reverse(x=fixed_noise, y=fixed_y, guidance=guidance, attn_temp=attn_temp)
        guided_samples[guidance] = samples
    tv.utils.save_image(samples, os.path.join(sample_dir, f'samples.png'), normalize=True, nrow=4)

In [8]:
# finally we denoise the samples
for p in model.parameters():
    p.requires_grad = False
    
# remember the loss is mean, whereas log prob is sum
lr = batch_size * img_size ** 2 * channel_size * noise_std ** 2
for guidance, sample in guided_samples.items():
    x = torch.clone(guided_samples[guidance]).detach()
    x.requires_grad = True
    with torch.autocast(device_type=device, dtype=torch.bfloat16):
        z, outputs, logdets = model(x, fixed_y)
    loss = model.get_loss(z, logdets)
    grad = torch.autograd.grad(loss, [x])[0]
    x.data = x.data - lr * grad
    samples = x
    print(f'denoising complete')
    tv.utils.save_image(samples, os.path.join(sample_dir, f'samples_denoised.png'), normalize=True, nrow=4)

denoising complete
